In [2]:
import matplotlib
matplotlib.use('Agg')
import pandas as pd
import seaborn as sns
import glob
import numpy as np
import pickle
# Some preprocessing utilities
from sklearn.cross_validation import train_test_split # Data splitting
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
# Model result function
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn import preprocessing
from sklearn.utils import shuffle
import itertools
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import normalize
sns.set_context('paper')
palette = sns.plt.cm.gray_r
# set masked values to plot red
palette.set_bad('r', 1.0)
%matplotlib inline

data_files = glob.glob("HH133*")#+ glob.glob("HH13*pks16*0and2*csv")
print data_files

names = ["d_p","tC_mean","tp_mean",'t_skew','t_kurtosis','t_cumsumx','t_sumvalues'
         ,'t_slope','t_per25','t_per75','t_var',"fC_mean","fp_mean",'f_skew'
         ,'f_kurtosis','f_cumsumx','f_sumvalues','f_slope','f_per25','f_per75','f_var']
target_names = ['Not RFI','RFI']

['HH1333134985_pksj1939-6342.1822.ms_all_data_baseline1and6.csv', 'HH1333305511_pksj1939-6342.1822.ms_all_data_baseline0and2.csv', 'HH1333305511_pksj1939-6342.1822.ms_all_data_baseline3and6.csv', 'HH1333305511_pks1613-586.1822.ms_all_data_baseline1and4.csv', 'HH1333305511_pksj1939-6342.1822.ms_all_data_baseline0and6.csv', 'HH1333305511_pks1613-586.1822.ms_all_data_baseline2and6.csv', 'HH1333305511_pks1613-586.1822.ms_all_data_baseline1and3.csv', 'HH1333305511_pks1613-586.1822.ms_all_data_baseline4and6.csv', 'HH1333134985_pks1613-586.1822.ms_all_data_baseline1and3.csv', 'HH1333305511_pks1613-586.1822.ms_all_data_baseline0and6.csv', 'HH1333134985_pks1613-586.1822.ms_all_data_baseline5and6.csv', 'HH1333305511_pks1613-586.1822.ms_all_data_baseline1and5.csv', 'HH1333305511_pksj1939-6342.1822.ms_all_data_baseline2and5.csv', 'HH1333134985_pks1613-586.1822.ms_all_data_baseline0and2.csv', 'HH1333305511_pksj1939-6342.1822.ms_all_data_baseline2and3.csv', 'HH1333134985_pksj1939-6342.1822.ms_all_da

/data/olorato/mywork/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
datasets = []
files = []
for filename in data_files:#[:15]:
    data = pd.DataFrame.from_csv(filename)
    if True:#len(np.where(data["rfi"].values == 1)[0])/float(len(data["rfi"])) < 0.4:
        try:
            for name in names:
                data[name] = normalize(data[name],norm='l2')[0]
            HH_X0 = data
            files.append(filename)
            del data
            data = pd.DataFrame.from_csv(filename.replace('HH','VV'))
            for name in names:
                data[name] = normalize(data[name],norm='l2')[0]
            VV_X0 = data
            del data
            data = pd.DataFrame.from_csv(filename.replace('HH','HV'))
            for name in names:
                data[name] = normalize(data[name],norm='l2')[0]
            HV_X0 = data
            del data


            x_train0HH, x_test0HH, y_train0HH, y_test0HH = train_test_split(HH_X0[names], HH_X0['rfi'], test_size=0.30,random_state=0)

            x_train0HV, x_test0HV, y_train0HV, y_test0HV = train_test_split(HV_X0[names], HV_X0['rfi'], test_size=0.30,random_state=0)

            x_train0VV, x_test0VV, y_train0VV, y_test0VV = train_test_split(VV_X0[names], VV_X0['rfi'], test_size=0.30,random_state=0)

            x_train0HH['rfi'] = y_train0HH
            x_train0VV['rfi'] = y_train0VV
            x_train0HV['rfi'] = y_train0HV

            HHrfc = GaussianNB()
            HHrfc.fit(x_train0HH[names],y_train0HH)
            HHy_pred_prob_rfc = HHrfc.predict_proba(x_test0HH)
            HHy_pred_rfc = HHrfc.predict(x_test0HH)

            VVrfc = GaussianNB()
            VVrfc.fit(x_train0VV[names],y_train0VV)
            VVy_pred_prob_rfc = VVrfc.predict_proba(x_test0VV)
            VVy_pred_rfc = VVrfc.predict(x_test0VV)

            HVrfc = GaussianNB()
            HVrfc.fit(x_train0HV[names],y_train0HV)
            HVy_pred_prob_rfc = HVrfc.predict_proba(x_test0HV)
            HVy_pred_rfc = HVrfc.predict(x_test0HV)

            def roc_plot(classifiers,subplot,model_names):
        #c = ['r','g','b']
                if subplot == True:
                    num = len(classifiers)
                    sns.plt.figure()
                    for i in range(num):
                        sns.plt.subplot(1,num,i+1)
                        clf = classifiers[i]
                        y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0, clf[:,1])
                        sns.plt.plot(y_roc_fpr, y_roc_tpr,label=model_names[i]+' AUC = %0.2f'% auc(y_roc_fpr, y_roc_tpr))
                        sns.plt.legend(loc='lower right',fontsize = 20 )
                        sns.plt.plot([0,1],[0,1],'r--')
                        sns.plt.xlim([-0.1,1.2])
                        sns.plt.ylim([-0.1,1.2])
                        sns.plt.ylabel('True Positive Rate',fontsize = 34 ,fontweight='bold')
                        sns.plt.xlabel('False Positive Rate',fontsize = 34 ,fontweight='bold')
                        sns.plt.xticks(size=18)
                        sns.plt.yticks(size=18)
                if subplot == False:
                    num = len(classifiers)
                    sns.plt.figure(figsize=(20,20))
                    for i in range(num):
                        clf = classifiers[i]
                        if 'VV' in model_names[i]:
                            y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0VV, clf[:,1])
                        elif 'HV' in model_names[i]:
                            y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0HV, clf[:,1])
                        else:
                            y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0HH, clf[:,1])
                        sns.plt.plot(y_roc_fpr, y_roc_tpr,label=model_names[i]+' AUC = %0.2f'% auc(y_roc_fpr, y_roc_tpr))
                        sns.plt.legend(loc='lower right',fontsize = 20 )
                        sns.plt.plot([0,1],[0,1],'r--')
                        sns.plt.xlim([-0.1,1.2])
                        sns.plt.ylim([-0.1,1.2])
                        sns.plt.ylabel('True Positive Rate',fontsize = 34 ,fontweight='bold')
                        sns.plt.xlabel('False Positive Rate',fontsize = 34 ,fontweight='bold')
                        sns.plt.xticks(size=18)
                        sns.plt.yticks(size=18)
                sns.plt.savefig('ROC_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.eps', format='eps', dpi=600)
                return
            model_names =['HH','VV','HV']
            roc_plot([HHy_pred_prob_rfc ,VVy_pred_prob_rfc,HVy_pred_prob_rfc],False,model_names)

            cnf_matrix = confusion_matrix(y_test0HH,HHy_pred_rfc)
            sns.plt.figure(figsize=(20,20))
            sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
            classes = target_names 
            #sns.plt.title(title)
            sns.plt.colorbar()
            tick_marks = np.arange(len(classes))
            sns.plt.xticks(tick_marks, classes, rotation=45)
            sns.plt.yticks(tick_marks, classes)

            #if normalize:
            #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
                #print("Normalized confusion matrix")
            #else:
            #    pass
                #print('Confusion matrix, without normalization')

            #print(cm)

            thresh = cnf_matrix.max() / 2.
            for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
                sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                         horizontalalignment="center",
                         color="white" if cnf_matrix[i, j] > thresh else "black")

            sns.plt.tight_layout()
            sns.plt.grid(False)
            sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
            sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
            sns.plt.xticks(size=18)
            sns.plt.yticks(size=18)
            sns.plt.savefig('HH_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.eps', format='eps', dpi=600)

            open_file = open('HH_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.txt','w')

            open_file.write('====================================================')
            open_file.write('Classification Report for HH')
            open_file.write('====================================================')
            open_file.write(classification_report(y_test0HH,HHy_pred_rfc,target_names=['Not RFI','RFI']))
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,HHy_pred_rfc)*100))
            open_file.close()

            cnf_matrix = confusion_matrix(y_test0HV,HVy_pred_rfc)
            sns.plt.figure(figsize=(20,20))
            sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
            classes = target_names 
            #sns.plt.title(title)
            sns.plt.colorbar()
            tick_marks = np.arange(len(classes))
            sns.plt.xticks(tick_marks, classes, rotation=45)
            sns.plt.yticks(tick_marks, classes)

            #if normalize:
            #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
                #print("Normalized confusion matrix")
            #else:
            #    pass
                #print('Confusion matrix, without normalization')

            #print(cm)

            thresh = cnf_matrix.max() / 2.
            for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
                sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                         horizontalalignment="center",
                         color="white" if cnf_matrix[i, j] > thresh else "black")

            sns.plt.tight_layout()
            sns.plt.grid(False)
            sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
            sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
            sns.plt.xticks(size=18)
            sns.plt.yticks(size=18)
            sns.plt.savefig('HV_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.eps', format='eps', dpi=600)

            open_file = open('HV_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.txt','w')

            open_file.write('====================================================')
            open_file.write('Classification Report for HV')
            open_file.write('====================================================')
            open_file.write(classification_report(y_test0HV,HVy_pred_rfc,target_names=['Not RFI','RFI']))
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,HVy_pred_rfc)*100))
            open_file.close()

            cnf_matrix = confusion_matrix(y_test0VV,VVy_pred_rfc)
            sns.plt.figure(figsize=(20,20))
            sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
            classes = target_names 
            #sns.plt.title(title)
            sns.plt.colorbar()
            tick_marks = np.arange(len(classes))
            sns.plt.xticks(tick_marks, classes, rotation=45)
            sns.plt.yticks(tick_marks, classes)

            #if normalize:
            #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
                #print("Normalized confusion matrix")
            #else:
            #    pass
                #print('Confusion matrix, without normalization')

            #print(cm)

            thresh = cnf_matrix.max() / 2.
            for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
                sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                         horizontalalignment="center",
                         color="white" if cnf_matrix[i, j] > thresh else "black")

            sns.plt.tight_layout()
            sns.plt.grid(False)
            sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
            sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
            sns.plt.xticks(size=18)
            sns.plt.yticks(size=18)
            sns.plt.savefig('VV_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.eps', format='eps', dpi=600)

            open_file = open('VV_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.txt','w')

            open_file.write('====================================================')
            open_file.write('Classification Report for VV')
            open_file.write('====================================================')
            open_file.write(classification_report(y_test0VV,VVy_pred_rfc,target_names=['Not RFI','RFI']))
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,VVy_pred_rfc)*100))
            open_file.close()

            #testVV = pd.DataFrame.from_csv('VV1333305511_pks1613-586.1822.ms_all_data_baseline0and1.csv')
            open_file = open('poltest_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.txt','w')

            open_file.write("HH")
            y_pred_rfc = HHrfc.predict(x_test0HH)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

            y_pred_rfc = HHrfc.predict(x_test0VV)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

            y_pred_rfc = HHrfc.predict(x_test0HV)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))


            open_file.write("VV")
            y_pred_rfc = VVrfc.predict(x_test0HH)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

            y_pred_rfc = VVrfc.predict(x_test0VV)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

            y_pred_rfc = VVrfc.predict(x_test0HV)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))

            open_file.write("HV")
            y_pred_rfc = HVrfc.predict(x_test0HH)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

            y_pred_rfc = HVrfc.predict(x_test0VV)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

            y_pred_rfc = HVrfc.predict(x_test0HV)
            open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))

            open_file.close()


            pickle.dump(HHrfc,open('HH_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.p','wb'))
            pickle.dump(HVrfc,open('HV_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.p','wb'))
            pickle.dump(VVrfc,open('VV_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_NB.p','wb'))
        except Exception as e:
            print e
            pass

/data/olorato/mywork/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/data/olorato/mywork/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/data/olorato/mywork/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contain

/data/olorato/mywork/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/data/olorato/mywork/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/data/olorato/mywork/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contain

In [ ]:
datasets = []
files = []
for filename in data_files:#[:15]:
    data = pd.DataFrame.from_csv(filename)
    if len(np.where(data["rfi"].values == 1)[0])/float(len(data["rfi"])) < 0.4:
        for name in names:
            data[name] = normalize(data[name],norm='l2')[0]
        HH_X0 = data
        files.append(filename)
        del data
        data = pd.DataFrame.from_csv(filename.replace('HH','VV'))
        for name in names:
            data[name] = normalize(data[name],norm='l2')[0]
        VV_X0 = data
        del data
        data = pd.DataFrame.from_csv(filename.replace('HH','HV'))
        for name in names:
            data[name] = normalize(data[name],norm='l2')[0]
        HV_X0 = data
        del data
        
        
        x_train0HH, x_test0HH, y_train0HH, y_test0HH = train_test_split(HH_X0[names], HH_X0['rfi'], test_size=0.30,random_state=0)

        x_train0HV, x_test0HV, y_train0HV, y_test0HV = train_test_split(HV_X0[names], HV_X0['rfi'], test_size=0.30,random_state=0)

        x_train0VV, x_test0VV, y_train0VV, y_test0VV = train_test_split(VV_X0[names], VV_X0['rfi'], test_size=0.30,random_state=0)

        x_train0HH['rfi'] = y_train0HH
        x_train0VV['rfi'] = y_train0VV
        x_train0HV['rfi'] = y_train0HV
        
        
        HHcv = StratifiedKFold(x_train0HH, n_folds=15,shuffle=True, random_state=0)
        VVcv = StratifiedKFold(x_train0VV, n_folds=15,shuffle=True, random_state=0)
        HVcv = StratifiedKFold(x_train0HV, n_folds=15,shuffle=True, random_state=0)
        rfc = RandomForestClassifier(random_state=0)
        grid = {'n_estimators': np.arange(1, 11)
                ,'criterion': ['gini', 'entropy']
                ,'max_features':['sqrt','log2']
                ,'max_depth':np.arange(2,11)
                ,"min_samples_split": np.arange(2, 11)
                ,"min_samples_leaf": np.arange(2, 11)
                ,"bootstrap": [True, False]
                ,'min_samples_split':np.arange(2,11)}

        HHrfc = RandomizedSearchCV(rfc, grid, cv=HHcv,n_iter=20)
        HHrfc.fit(x_train0HH[names],y_train0HH)
        VVrfc = RandomizedSearchCV(rfc, grid, cv=VVcv,n_iter=20)
        VVrfc.fit(x_train0VV[names],y_train0VV)
        HVrfc = RandomizedSearchCV(rfc, grid, cv=HVcv,n_iter=20)
        HVrfc.fit(x_train0HV[names],y_train0HV)
        
        
        HHrfc = HHrfc.best_estimator_
        HHrfc.fit(x_train0HH[names],y_train0HH)
        HHy_pred_prob_rfc = HHrfc.predict_proba(x_test0HH)
        HHy_pred_rfc = HHrfc.predict(x_test0HH)
        
        VVrfc = VVrfc.best_estimator_
        VVrfc.fit(x_train0VV[names],y_train0VV)
        VVy_pred_prob_rfc = VVrfc.predict_proba(x_test0VV)
        VVy_pred_rfc = VVrfc.predict(x_test0VV)

        HVrfc = HHrfc.best_estimator_
        HVrfc.fit(x_train0HV[names],y_train0HV)
        HVy_pred_prob_rfc = HVrfc.predict_proba(x_test0HV)
        HVy_pred_rfc = HVrfc.predict(x_test0HV)
        
        def roc_plot(classifiers,subplot,model_names):
    #c = ['r','g','b']
            if subplot == True:
                num = len(classifiers)
                sns.plt.figure()
                for i in range(num):
                    sns.plt.subplot(1,num,i+1)
                    clf = classifiers[i]
                    y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0, clf[:,1])
                    sns.plt.plot(y_roc_fpr, y_roc_tpr,label=model_names[i]+' AUC = %0.2f'% auc(y_roc_fpr, y_roc_tpr))
                    sns.plt.legend(loc='lower right',fontsize = 20 )
                    sns.plt.plot([0,1],[0,1],'r--')
                    sns.plt.xlim([-0.1,1.2])
                    sns.plt.ylim([-0.1,1.2])
                    sns.plt.ylabel('True Positive Rate',fontsize = 34 ,fontweight='bold')
                    sns.plt.xlabel('False Positive Rate',fontsize = 34 ,fontweight='bold')
                    sns.plt.xticks(size=18)
                    sns.plt.yticks(size=18)
            if subplot == False:
                num = len(classifiers)
                sns.plt.figure(figsize=(10,10))
                for i in range(num):
                    clf = classifiers[i]
                    if 'VV' in model_names[i]:
                        y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0VV, clf[:,1])
                    elif 'HV' in model_names[i]:
                        y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0HV, clf[:,1])
                    else:
                        y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0HH, clf[:,1])
                    sns.plt.plot(y_roc_fpr, y_roc_tpr,label=model_names[i]+' AUC = %0.2f'% auc(y_roc_fpr, y_roc_tpr))
                    sns.plt.legend(loc='lower right',fontsize = 20 )
                    sns.plt.plot([0,1],[0,1],'r--')
                    sns.plt.xlim([-0.1,1.2])
                    sns.plt.ylim([-0.1,1.2])
                    sns.plt.ylabel('True Positive Rate',fontsize = 34 ,fontweight='bold')
                    sns.plt.xlabel('False Positive Rate',fontsize = 34 ,fontweight='bold')
                    sns.plt.xticks(size=18)
                    sns.plt.yticks(size=18)
            sns.plt.savefig('ROC_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.eps', format='eps', dpi=600)
            return
        model_names =['HH','VV','HV']
        roc_plot([HHy_pred_prob_rfc ,VVy_pred_prob_rfc,HVy_pred_prob_rfc],False,model_names)
        
        cnf_matrix = confusion_matrix(y_test0HH,HHy_pred_rfc)
        sns.plt.figure(figsize=(20,20))
        sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
        classes = target_names 
        #sns.plt.title(title)
        sns.plt.colorbar()
        tick_marks = np.arange(len(classes))
        sns.plt.xticks(tick_marks, classes, rotation=45)
        sns.plt.yticks(tick_marks, classes)

        #if normalize:
        #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            #print("Normalized confusion matrix")
        #else:
        #    pass
            #print('Confusion matrix, without normalization')

        #print(cm)

        thresh = cnf_matrix.max() / 2.
        for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
            sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                     horizontalalignment="center",
                     color="white" if cnf_matrix[i, j] > thresh else "black")

        sns.plt.tight_layout()
        sns.plt.grid(False)
        sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
        sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
        sns.plt.xticks(size=18)
        sns.plt.yticks(size=18)
        sns.plt.savefig('HH_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.eps', format='eps', dpi=600)
        
        open_file = open('HH_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.txt','w')
        
        open_file.write('====================================================')
        open_file.write('Classification Report for HH')
        open_file.write('====================================================')
        open_file.write(classification_report(y_test0HH,HHy_pred_rfc,target_names=['Not RFI','RFI']))
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,HHy_pred_rfc)*100))
        open_file.close()
        
        cnf_matrix = confusion_matrix(y_test0HV,HVy_pred_rfc)
        sns.plt.figure(figsize=(20,20))
        sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
        classes = target_names 
        #sns.plt.title(title)
        sns.plt.colorbar()
        tick_marks = np.arange(len(classes))
        sns.plt.xticks(tick_marks, classes, rotation=45)
        sns.plt.yticks(tick_marks, classes)

        #if normalize:
        #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            #print("Normalized confusion matrix")
        #else:
        #    pass
            #print('Confusion matrix, without normalization')

        #print(cm)

        thresh = cnf_matrix.max() / 2.
        for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
            sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                     horizontalalignment="center",
                     color="white" if cnf_matrix[i, j] > thresh else "black")

        sns.plt.tight_layout()
        sns.plt.grid(False)
        sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
        sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
        sns.plt.xticks(size=18)
        sns.plt.yticks(size=18)
        sns.plt.savefig('HV_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.eps', format='eps', dpi=600)
        
        open_file = open('HV_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.txt','w')
        
        open_file.write('====================================================')
        open_file.write('Classification Report for HV')
        open_file.write('====================================================')
        open_file.write(classification_report(y_test0HV,HVy_pred_rfc,target_names=['Not RFI','RFI']))
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,HVy_pred_rfc)*100))
        open_file.close()
        
        cnf_matrix = confusion_matrix(y_test0VV,VVy_pred_rfc)
        sns.plt.figure(figsize=(20,20))
        sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
        classes = target_names 
        #sns.plt.title(title)
        sns.plt.colorbar()
        tick_marks = np.arange(len(classes))
        sns.plt.xticks(tick_marks, classes, rotation=45)
        sns.plt.yticks(tick_marks, classes)

        #if normalize:
        #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            #print("Normalized confusion matrix")
        #else:
        #    pass
            #print('Confusion matrix, without normalization')

        #print(cm)

        thresh = cnf_matrix.max() / 2.
        for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
            sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                     horizontalalignment="center",
                     color="white" if cnf_matrix[i, j] > thresh else "black")

        sns.plt.tight_layout()
        sns.plt.grid(False)
        sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
        sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
        sns.plt.xticks(size=18)
        sns.plt.yticks(size=18)
        sns.plt.savefig('VV_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.eps', format='eps', dpi=600)
        
        open_file = open('VV_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.txt','w')
        
        open_file.write('====================================================')
        open_file.write('Classification Report for VV')
        open_file.write('====================================================')
        open_file.write(classification_report(y_test0VV,VVy_pred_rfc,target_names=['Not RFI','RFI']))
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,VVy_pred_rfc)*100))
        open_file.close()
        
        #testVV = pd.DataFrame.from_csv('VV1333305511_pks1613-586.1822.ms_all_data_baseline0and1.csv')
        open_file = open('poltest_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.txt','w')

        open_file.write("HH")
        y_pred_rfc = HHrfc.predict(x_test0HH)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

        y_pred_rfc = HHrfc.predict(x_test0VV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

        y_pred_rfc = HHrfc.predict(x_test0HV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))


        open_file.write("VV")
        y_pred_rfc = VVrfc.predict(x_test0HH)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

        y_pred_rfc = VVrfc.predict(x_test0VV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

        y_pred_rfc = VVrfc.predict(x_test0HV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))

        open_file.write("HV")
        y_pred_rfc = HVrfc.predict(x_test0HH)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

        y_pred_rfc = HVrfc.predict(x_test0VV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

        y_pred_rfc = HVrfc.predict(x_test0HV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))
        
        open_file.close()

        open_file = open('frank_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_RFC.txt','w')

        importances = HHrfc.feature_importances_
        std = np.std([tree.feature_importances_ for tree in HHrfc.estimators_],
                     axis=0)
        indices = np.argsort(importances)[::-1]

        feature_space= []
        # Print the feature ranking
        open_file.write("Feature ranking: HH")

        for f in range(x_train0HH[names].shape[1]):
            open_file.write("%d. feature %s (%f)" % (f + 1, names[indices[f]], importances[indices[f]]))
            feature_space.append(names[indices[f]])

            
        importances = HVrfc.feature_importances_
        std = np.std([tree.feature_importances_ for tree in HVfc.estimators_],
                     axis=0)
        indices = np.argsort(importances)[::-1]

        feature_space= []
        # Print the feature ranking
        open_file.write("Feature ranking: HV")

        for f in range(x_train0HH[names].shape[1]):
            open_file.write("%d. feature %s (%f)" % (f + 1, names[indices[f]], importances[indices[f]]))
            feature_space.append(names[indices[f]])
        
        importances = VVrfc.feature_importances_
        std = np.std([tree.feature_importances_ for tree in VVrfc.estimators_],
                     axis=0)
        indices = np.argsort(importances)[::-1]

        feature_space= []
        # Print the feature ranking
        open_file.write("Feature ranking: VV")

        for f in range(x_train0HH[names].shape[1]):
            open_file.write("%d. feature %s (%f)" % (f + 1, names[indices[f]], importances[indices[f]]))
            feature_space.append(names[indices[f]])
        open_file.close()


datasets = []
files = []
for filename in data_files:#[:15]:
    data = pd.DataFrame.from_csv(filename)
    if len(np.where(data["rfi"].values == 1)[0])/float(len(data["rfi"])) < 0.4:
        for name in names:
            data[name] = normalize(data[name],norm='l2')[0]
        HH_X0 = data
        files.append(filename)
        del data
        data = pd.DataFrame.from_csv(filename.replace('HH','VV'))
        for name in names:
            data[name] = normalize(data[name],norm='l2')[0]
        VV_X0 = data
        del data
        data = pd.DataFrame.from_csv(filename.replace('HH','HV'))
        for name in names:
            data[name] = normalize(data[name],norm='l2')[0]
        HV_X0 = data
        del data
        
        
        x_train0HH, x_test0HH, y_train0HH, y_test0HH = train_test_split(HH_X0[names], HH_X0['rfi'], test_size=0.30,random_state=0)

        x_train0HV, x_test0HV, y_train0HV, y_test0HV = train_test_split(HV_X0[names], HV_X0['rfi'], test_size=0.30,random_state=0)

        x_train0VV, x_test0VV, y_train0VV, y_test0VV = train_test_split(VV_X0[names], VV_X0['rfi'], test_size=0.30,random_state=0)

        x_train0HH['rfi'] = y_train0HH
        x_train0VV['rfi'] = y_train0VV
        x_train0HV['rfi'] = y_train0HV
        
        HHcv = StratifiedKFold(x_train0HH, n_folds=15,shuffle=True, random_state=0)
        VVcv = StratifiedKFold(x_train0VV, n_folds=15,shuffle=True, random_state=0)
        HVcv = StratifiedKFold(x_train0HV, n_folds=15,shuffle=True, random_state=0)
        rfc = KNeighborsClassifier(algorithm='auto')
        grid = {"n_neighbors": np.arange(1,20)
                ,'weights ': ['uniform','distance']
                ,'p':np.arange(1,10)}

        HHrfc = RandomizedSearchCV(rfc, grid, cv=HHcv,n_iter=20)
        HHrfc.fit(x_train0HH[names],y_train0HH)
        VVrfc = RandomizedSearchCV(rfc, grid, cv=VVcv,n_iter=20)
        VVrfc.fit(x_train0VV[names],y_train0VV)
        HVrfc = RandomizedSearchCV(rfc, grid, cv=HVcv,n_iter=20)
        HVrfc.fit(x_train0HV[names],y_train0HV)
        
        
        HHrfc = HHrfc.best_estimator_
        HHrfc.fit(x_train0HH[names],y_train0HH)
        HHy_pred_prob_rfc = HHrfc.predict_proba(x_test0HH)
        HHy_pred_rfc = HHrfc.predict(x_test0HH)
        
        VVrfc = VVrfc.best_estimator_
        VVrfc.fit(x_train0VV[names],y_train0VV)
        VVy_pred_prob_rfc = VVrfc.predict_proba(x_test0VV)
        VVy_pred_rfc = VVrfc.predict(x_test0VV)

        HVrfc = HHrfc.best_estimator_
        HVrfc.fit(x_train0HV[names],y_train0HV)
        HVy_pred_prob_rfc = HVrfc.predict_proba(x_test0HV)
        HVy_pred_rfc = HVrfc.predict(x_test0HV)
        
        def roc_plot(classifiers,subplot,model_names):
    #c = ['r','g','b']
            if subplot == True:
                num = len(classifiers)
                sns.plt.figure()
                for i in range(num):
                    sns.plt.subplot(1,num,i+1)
                    clf = classifiers[i]
                    y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0, clf[:,1])
                    sns.plt.plot(y_roc_fpr, y_roc_tpr,label=model_names[i]+' AUC = %0.2f'% auc(y_roc_fpr, y_roc_tpr))
                    sns.plt.legend(loc='lower right',fontsize = 20 )
                    sns.plt.plot([0,1],[0,1],'r--')
                    sns.plt.xlim([-0.1,1.2])
                    sns.plt.ylim([-0.1,1.2])
                    sns.plt.ylabel('True Positive Rate',fontsize = 34 ,fontweight='bold')
                    sns.plt.xlabel('False Positive Rate',fontsize = 34 ,fontweight='bold')
                    sns.plt.xticks(size=18)
                    sns.plt.yticks(size=18)
            if subplot == False:
                num = len(classifiers)
                sns.plt.figure(figsize=(20,20))
                for i in range(num):
                    clf = classifiers[i]
                    if 'VV' in model_names[i]:
                        y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0VV, clf[:,1])
                    elif 'HV' in model_names[i]:
                        y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0HV, clf[:,1])
                    else:
                        y_roc_fpr, y_roc_tpr, y_roc_thresholds = roc_curve(y_test0HH, clf[:,1])
                    sns.plt.plot(y_roc_fpr, y_roc_tpr,label=model_names[i]+' AUC = %0.2f'% auc(y_roc_fpr, y_roc_tpr))
                    sns.plt.legend(loc='lower right',fontsize = 20 )
                    sns.plt.plot([0,1],[0,1],'r--')
                    sns.plt.xlim([-0.1,1.2])
                    sns.plt.ylim([-0.1,1.2])
                    sns.plt.ylabel('True Positive Rate',fontsize = 34 ,fontweight='bold')
                    sns.plt.xlabel('False Positive Rate',fontsize = 34 ,fontweight='bold')
                    sns.plt.xticks(size=18)
                    sns.plt.yticks(size=18)
            sns.plt.savefig('ROC_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.eps', format='eps', dpi=600)
            return
        model_names =['HH','VV','HV']
        roc_plot([HHy_pred_prob_rfc ,VVy_pred_prob_rfc,HVy_pred_prob_rfc],False,model_names)
        
        cnf_matrix = confusion_matrix(y_test0HH,HHy_pred_rfc)
        sns.plt.figure(figsize=(20,20))
        sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
        classes = target_names 
        #sns.plt.title(title)
        sns.plt.colorbar()
        tick_marks = np.arange(len(classes))
        sns.plt.xticks(tick_marks, classes, rotation=45)
        sns.plt.yticks(tick_marks, classes)

        #if normalize:
        #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            #print("Normalized confusion matrix")
        #else:
        #    pass
            #print('Confusion matrix, without normalization')

        #print(cm)

        thresh = cnf_matrix.max() / 2.
        for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
            sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                     horizontalalignment="center",
                     color="white" if cnf_matrix[i, j] > thresh else "black")

        sns.plt.tight_layout()
        sns.plt.grid(False)
        sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
        sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
        sns.plt.xticks(size=18)
        sns.plt.yticks(size=18)
        sns.plt.savefig('HH_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.eps', format='eps', dpi=600)
        
        open_file = open('HH_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.txt','w')
        
        open_file.write('====================================================')
        open_file.write('Classification Report for HH')
        open_file.write('====================================================')
        open_file.write(classification_report(y_test0HH,HHy_pred_rfc,target_names=['Not RFI','RFI']))
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,HHy_pred_rfc)*100))
        open_file.close()
        
        cnf_matrix = confusion_matrix(y_test0HV,HVy_pred_rfc)
        sns.plt.figure(figsize=(20,20))
        sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
        classes = target_names 
        #sns.plt.title(title)
        sns.plt.colorbar()
        tick_marks = np.arange(len(classes))
        sns.plt.xticks(tick_marks, classes, rotation=45)
        sns.plt.yticks(tick_marks, classes)

        #if normalize:
        #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            #print("Normalized confusion matrix")
        #else:
        #    pass
            #print('Confusion matrix, without normalization')

        #print(cm)

        thresh = cnf_matrix.max() / 2.
        for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
            sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                     horizontalalignment="center",
                     color="white" if cnf_matrix[i, j] > thresh else "black")

        sns.plt.tight_layout()
        sns.plt.grid(False)
        sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
        sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
        sns.plt.xticks(size=18)
        sns.plt.yticks(size=18)
        sns.plt.savefig('HV_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.eps', format='eps', dpi=600)
        
        open_file = open('HV_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.txt','w')
        
        open_file.write('====================================================')
        open_file.write('Classification Report for HV')
        open_file.write('====================================================')
        open_file.write(classification_report(y_test0HV,HVy_pred_rfc,target_names=['Not RFI','RFI']))
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,HVy_pred_rfc)*100))
        open_file.close()
        
        cnf_matrix = confusion_matrix(y_test0VV,VVy_pred_rfc)
        sns.plt.figure(figsize=(20,20))
        sns.plt.imshow(cnf_matrix, interpolation='nearest', cmap=sns.plt.cm.Blues)
        classes = target_names 
        #sns.plt.title(title)
        sns.plt.colorbar()
        tick_marks = np.arange(len(classes))
        sns.plt.xticks(tick_marks, classes, rotation=45)
        sns.plt.yticks(tick_marks, classes)

        #if normalize:
        #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            #print("Normalized confusion matrix")
        #else:
        #    pass
            #print('Confusion matrix, without normalization')

        #print(cm)

        thresh = cnf_matrix.max() / 2.
        for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
            sns.plt.text(j, i, round(cnf_matrix[i, j],3),
                     horizontalalignment="center",
                     color="white" if cnf_matrix[i, j] > thresh else "black")

        sns.plt.tight_layout()
        sns.plt.grid(False)
        sns.plt.ylabel('True label',fontsize = 34 ,fontweight='bold')
        sns.plt.xlabel('Predicted label',fontsize = 34 ,fontweight='bold')
        sns.plt.xticks(size=18)
        sns.plt.yticks(size=18)
        sns.plt.savefig('VV_Confusion_matrix_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.eps', format='eps', dpi=600)
        
        open_file = open('VV_class_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.txt','w')
        
        open_file.write('====================================================')
        open_file.write('Classification Report for VV')
        open_file.write('====================================================')
        open_file.write(classification_report(y_test0VV,VVy_pred_rfc,target_names=['Not RFI','RFI']))
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,VVy_pred_rfc)*100))
        open_file.close()
        
        #testVV = pd.DataFrame.from_csv('VV1333305511_pks1613-586.1822.ms_all_data_baseline0and1.csv')
        open_file = open('poltest_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.txt','w')

        open_file.write("HH")
        y_pred_rfc = HHrfc.predict(x_test0HH)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

        y_pred_rfc = HHrfc.predict(x_test0VV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

        y_pred_rfc = HHrfc.predict(x_test0HV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))


        open_file.write("VV")
        y_pred_rfc = VVrfc.predict(x_test0HH)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

        y_pred_rfc = VVrfc.predict(x_test0VV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

        y_pred_rfc = VVrfc.predict(x_test0HV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))

        open_file.write("HV")
        y_pred_rfc = HVrfc.predict(x_test0HH)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HH,y_pred_rfc)*100))

        y_pred_rfc = HVrfc.predict(x_test0VV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0VV,y_pred_rfc)*100))

        y_pred_rfc = HVrfc.predict(x_test0HV)
        open_file.write('\n The model is %f accurate' %(accuracy_score(y_test0HV,y_pred_rfc)*100))
        
        open_file.close()
        
        pickle.dump(HHrfc,open('HH_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.p','wb'))
        pickle.dump(HVrfc,open('HV_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.p','wb'))
        pickle.dump(VVrfc,open('VV_Same_source_'+filename.split("_")[1].split('.')[0]+'_and_baseline_'+filename.split('baseline')[-1].split('.')[0]+'_cross_validation_KNN.p','wb'))


print "done"